1920 - Ungen förbjöds att vara med pga sin roll och inblandning i första världskriget.  
  

1984 - Ungern bojkottade OS 1984 i Los Angeles, tillsammans med andra öststatländer, som svar på den USA-ledda bojkotten av OS i Moskva 1980. Bojkotten var en politisk handling som genomfördes på Sovjetunionens begäran. 
Orsak: Bojkotten var en vedergällning för den bojkott som USA och andra länder genomförde mot OS i Moskva 1980.
Beslut: Ungern deltog inte eftersom de, likt andra socialistiska länder, följde Sovjetunionens beslut.  
  
(1980 - USA ledde en bojkott av de Olympiska spelen i Moskva som en protest mot Sovjets invasion av Afghanistan. Ytterligare 60 länder deltog i bojkotten, bland annat Kina.)  
  
1984 - Som en följd av det Kalla kriget bojkottade Sovjet och många andra länder i östblocket de Olympiska spelen i Los Angeles.  

In [ ]:
import pandas as pd

# Ladda in data
athletes = pd.read_csv("athlete_events.csv")
noc = pd.read_csv("noc_regions.csv")

# Slå ihop dataset för att få regionnamn
data = pd.merge(athletes, noc, on="NOC", how="left")

# Filtrera på Ungern
hungary = data[data["region"] == "Hungary"]

sweden = data[data["region"] == "Sweden"]

In [ ]:
data

In [ ]:
hungary.info()
hungary.describe()

In [ ]:
# --- a. Hur många länder som är med i datan ---
num_countries = data.nunique()
print(f"a) Antal länder: {num_countries}")

In [ ]:
# --- b. Vilka länder är med (förkortningarna räcker) ---
countries = athletes['NOC'].unique()
print("b) Länder (NOC-förkortningar):")
print(countries)

In [ ]:
# --- c. Vilka sporter är med ---
sports = athletes['Sport'].unique()
print(f"c) Antal sporter: {len(sports)}")
print(sports)

In [ ]:
# --- d. Vilka medaljtyper finns det ---
medals = athletes['Medal'].dropna().unique()
print(f"d) Medaljtyper: {medals}")

In [ ]:
# --- e. Statistik för åldern ---
age_stats = athletes['Age'].describe()
print("e) Statistik för ålder:")
print(age_stats[['mean', '50%', 'min', 'max', 'std']])

In [ ]:
# --- f. Egna frågor ---
# Exempel: Vilket år hade flest deltagare?
participants_per_year = athletes.groupby('Year')['ID'].nunique()
print("f) År med flest deltagare:")
print(participants_per_year.sort_values(ascending=False).head())

In [ ]:
# --- g. Diagram över könsfördelningen ---
gender_counts = athletes['Sex'].value_counts()
plt.figure(figsize=(5,5))
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Könsfördelning bland deltagare")
plt.ylabel("")  # ta bort y-etikett
plt.show()

In [ ]:
gender_counts = hungary['Sex'].value_counts()
plt.figure(figsize=(5,5))
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Könsfördelning bland deltagare")
plt.ylabel("")  # ta bort y-etikett
plt.show()

In [ ]:
gender_counts = sweden['Sex'].value_counts()
plt.figure(figsize=(5,5))
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title("Könsfördelning bland deltagare")
plt.ylabel("")  # ta bort y-etikett
plt.show()

In [ ]:
medal_counts = athletes.dropna(subset=['Medal']).groupby('NOC')['Medal'].count().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
medal_counts.plot(kind='bar', color='skyblue')
plt.title("Topp 10 länder med flest medaljer")
plt.xlabel("Land (NOC)")
plt.ylabel("Antal medaljer")
plt.show()

Antal ungerska deltagare per OS

In [ ]:
yearly_counts = hungary.groupby('Year')['ID'].nunique()
yearly_counts.plot(kind='line', marker='o')
plt.title("Antal ungerska deltagare per OS")
plt.xlabel("År")
plt.ylabel("Antal deltagare")
plt.show()

Antal svenska deltagare per OS

In [ ]:
yearly_counts = sweden.groupby('Year')['ID'].nunique()
yearly_counts.plot(kind='line', marker='o')
plt.title("Antal svenska deltagare per OS")
plt.xlabel("År")
plt.ylabel("Antal deltagare")
plt.show()

Antal medaljer 

In [ ]:
hungary[hungary['Medal'] == 0].shape[0]
hungary['Medal'].value_counts() 

Antal medaljer per år

In [ ]:
medals = hungary[hungary['Medal'].notna()]
medal_per_year = medals.groupby('Year')['Medal'].count()

medal_per_year.plot(kind='bar')
plt.title("Antal medaljer Ungern har tagit per år")
plt.xlabel("År")
plt.ylabel("Antal medaljer")
plt.show()